## Prepare input data

In [5]:
%%bash
mkdir -p ../data/fetchngs-input
cat << EOF > ../data/fetchngs-input/fetchngs-samplesheet.csv
SRR6848182
SRR12235536
SRR12235529
EOF
nextflow run nf-core/fetchngs -profile docker --outdir ../data/fetchngs-output --input ../data/fetchngs-input/fetchngs-samplesheet.csv

N E X T F L O W  ~  version 22.10.1
Pulling nf-core/fetchngs ...
 downloaded from https://github.com/nf-core/fetchngs.git
Launching `https://github.com/nf-core/fetchngs` [goofy_lorenz] DSL2 - revision: 084e5ef303 [master]


------------------------------------------------------
                                        ,--./,-.
        ___     __   __   __   ___     /,-._.--~'
  |\ | |__  __ /  ` /  \ |__) |__         }  {
  | \| |       \__, \__/ |  \ |___     \`-._,-`-,
                                        `._,._,'
  nf-core/fetchngs v1.9-g084e5ef
------------------------------------------------------
Core Nextflow options
  revision       : master
  runName        : goofy_lorenz
  containerEngine: docker
  launchDir      : /workspaces/nf-core-shapeseq/notebooks
  workDir        : /workspaces/nf-core-shapeseq/notebooks/work
  projectDir     : /home/gitpod/.nextflow/assets/nf-core/fetchngs
  userName       : gitpod
  profile        : docker
  configFiles    : /home/gitpod/.nextflow/a

CalledProcessError: Command 'b'mkdir -p ../data/fetchngs-input\ncat << EOF > ../data/fetchngs-input/fetchngs-samplesheet.csv\nSRR6848182\nSRR12235536\nSRR12235529\nEOF\nnextflow run nf-core/fetchngs -profile docker --outdir ../data/fetchngs-output --input ../data/fetchngs-input/fetchngs-samplesheet.csv\n'' returned non-zero exit status 1.